# Question Generation

In [1]:
import sys
import pandas as pd
from llama_index.core.evaluation import DatasetGenerator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response

In [2]:
import nest_asyncio

nest_asyncio.apply()

Download Data

In [11]:
pwd

'/home/johny02/projects/local-rag-chat'

In [3]:
cd /home/johny02/projects/local-rag-chat

/home/johny02/projects/local-rag-chat


/home/johny02/anaconda3/envs/local_rag_chat/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
mkdir -p 'data/paul_graham/'

In [5]:
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-12-16 15:06:42--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.04s   

2024-12-16 15:06:43 (1.64 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



Load Data

In [135]:
reader = SimpleDirectoryReader("./local_rag_chat/eval/data/paul_graham")
documents = reader.load_data()

In [9]:
from local_rag_chat.core.llms.ollama import OllamaModel
llm = OllamaModel(model='llama3.2')

2024-12-18 15:21:27,647 [INFO] app_logger: Model llama3.2 does not exist. Pulling model...
2024-12-18 15:21:30,014 [INFO] app_logger: Loaded Ollama model: llama3.2


In [ ]:
data_generator = DatasetGenerator.from_documents(documents, llm)

In [ ]:
eval_questions = data_generator.generate_questions_from_nodes()

In [16]:
with open('local_rag_chat/eval/data/paul_graham/questions.txt', 'w') as f:
    for question in eval_questions:
        f.write(f'{question.strip()}\n')

In [7]:
with open('local_rag_chat/eval/data/paul_graham/questions.txt', 'r') as f:
    questions = f.readlines()
    eval_questions = [line.rstrip() for line in questions]

# Retrieval Evaluation

**Hit Rate:**

Hit rate calculates the fraction of queries where the correct answer is found within the top-k retrieved documents. In simpler terms, it’s about how often our system gets it right within the top few guesses.

**Mean Reciprocal Rank (MRR):**

For each query, MRR evaluates the system’s accuracy by looking at the rank of the highest-placed relevant document. Specifically, it’s the average of the reciprocals of these ranks across all the queries. So, if the first relevant document is the top result, the reciprocal rank is 1; if it’s second, the reciprocal rank is 1/2, and so on.

Let's check these metrics to check the performance of out retriever.

## Base Retriever

In [51]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex

In [136]:
Settings.embed_model = embed_model

In [137]:
node_parser = SentenceSplitter(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)

In [138]:
len(nodes)

65

In [139]:
vector_index = VectorStoreIndex(nodes)
retriever = vector_index.as_retriever(similarity_top_k=5)

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00,  1.13it/s]


In [140]:
retrieved_nodes = retriever.retrieve("What was the primary reason you stopped using your time, when writing short stories and programming, after the introduction of microcomputers?")

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 27.50it/s]


In [141]:
from llama_index.core.response.notebook_utils import display_source_node

for node in retrieved_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** 3458ff03-e8b2-4275-beff-eda0af5da61c<br>**Similarity:** 0.7235259160039743<br>**Text:** What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in ...<br>

**Node ID:** 742915ba-3ca0-4606-a3d4-9473b4c4ae15<br>**Similarity:** 0.7182219860425836<br>**Text:** I was puzzled by the 1401. I couldn't figure out what to do with it. And in retrospect there's not much I could have done with it. The only form of input to programs was data stored on punched cards, and I didn't have any data stored on punched cards. The only other option was to do things that didn't rely on any input, like calculate approximations of pi, but I didn't know enough math to do anything interesting of that type. So I'm not surprised I can't remember any programs I wrote, because they can't have done much. My clearest memory is of the moment I learned it was possible for programs not to terminate, when one of mine didn't. On a machine without time-sharing, this was a social as well as a technical error, as the data center manager's expression made clear.

With microcomputers, everything changed. Now you could have a computer sitting right in front of you, on a desk, that could respond to your keystrokes as it was running instead of just churning through a stack of punch...<br>

**Node ID:** 85995b93-ee69-4f4f-8aec-2eca7d17fe64<br>**Similarity:** 0.7055373393296052<br>**Text:** I remember vividly how impressed and envious I felt watching him sitting in front of it, typing programs right into the computer.

Computers were expensive in those days and it took me years of nagging before I convinced my father to buy one, a TRS-80, in about 1980. The gold standard then was the Apple II, but a TRS-80 was good enough. This was when I really started programming. I wrote simple games, a program to predict how high my model rockets would fly, and a word processor that my father used to write at least one book. There was only room in memory for about 2 pages of text, so he'd write 2 pages at a time and then print them out, but it was a lot better than a typewriter.

Though I liked programming, I didn't plan to study it in college. In college I was going to study philosophy, which sounded much more powerful. It seemed, to my naive high school self, to be the study of the ultimate truths, compared to which the things studied in other fields would be mere domain knowledg...<br>

**Node ID:** 8ea39fa0-52db-40b1-89b2-34796b6eac28<br>**Similarity:** 0.6816770476743171<br>**Text:** What were the two main things the author worked on before college?
How did the author describe their early attempts at writing short stories?
What type of computer did the author first work on for programming?
What language did the author use for programming on the IBM 1401?
What was the author's experience with programming on the 1401?
What type of computer did the author eventually get for themselves?
What was the author's initial plan for college?
What made the author change their mind about studying philosophy?
What sparked the author's interest in AI?
What did the author realize about AI during their first year of grad school?
What were the two art schools that the author applied to?
How did the author end up at RISD?
What was the purpose of the foundation classes at RISD?
How did the author manage to pass the entrance exam for the Accademia di Belli Arti?
What was the arrangement between the students and faculty at the Accademia?
What was the author's experience painting still...<br>

**Node ID:** 81d23718-4cc0-4a2e-8076-e333b7145318<br>**Similarity:** 0.6689068279918875<br>**Text:** Now lots of startups get their initial set of customers almost entirely from among their batchmates.

I had not originally intended YC to be a full-time job. I was going to do three things: hack, write essays, and work on YC. As YC grew, and I grew more excited about it, it started to take up a lot more than a third of my attention. But for the first few years I was still able to work on other things.

In the summer of 2006, Robert and I started working on a new version of Arc. This one was reasonably fast, because it was compiled into Scheme. To test this new Arc, I wrote Hacker News in it. It was originally meant to be a news aggregator for startup founders and was called Startup News, but after a few months I got tired of reading about nothing but startups. Plus it wasn't startup founders we wanted to reach. It was future startup founders. So I changed the name to Hacker News and the topic to whatever engaged one's intellectual curiosity.

HN was no doubt good for YC, but it was ...<br>

In [147]:
qa_dataset_base = generate_question_context_pairs(
    nodes, num_questions_per_chunk=2
)

100%|███████████████████████████████████████████| 65/65 [26:24<00:00, 24.38s/it]


In [149]:
qa_dataset_base.queries = {key:value for key, value in qa_dataset_base.queries.items() if "Here are" not in value}

In [75]:
qa_dataset.queries = {key:value for key, value in qa_dataset.queries.items() if "Here are" not in value}

In [142]:
from llama_index.core.evaluation import RetrieverEvaluator
metrics = ["hit_rate", "mrr"]
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=retriever
)

In [150]:
list(qa_dataset_base.queries.items())[0]

('ea82b3b5-15c0-4d08-b9ae-06e6a23fabc2',
 'What was one of the main challenges you faced when working with the IBM 1401 computer, and how did it affect your programming endeavors?')

In [145]:
qa_dataset_base.relevant_docs['fbefe910-f921-4e08-a83b-c55de2197952']

['b1e946c3-84f2-4133-aa23-ef693ba9ed31']

In [146]:
qa_dataset_base.corpus['b1e946c3-84f2-4133-aa23-ef693ba9ed31']

'What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then stack

In [151]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset_base.queries.items())[0]
sample_expected = qa_dataset_base.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 26.45it/s]

Query: What was one of the main challenges you faced when working with the IBM 1401 computer, and how did it affect your programming endeavors?
Metrics: {'hit_rate': 1.0, 'mrr': 0.5}



In [152]:
# try it out on an entire dataset
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset_base)

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 26.74it/s]


In [153]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

In [154]:
display_results("base eval", eval_results)

,retrievers,hit_rate,mrr
0,base eval,0.769231,0.470513


## HybridRetriever

In [8]:
from local_rag_chat.core.loaders.simple_loader import SimpleLoader
from local_rag_chat.core.embeddings.embedding_manager import EmbeddingManager

embed_model = EmbeddingManager(model='BAAI/bge-small-en-v1.5').get_embedding()
loader = SimpleLoader(embed_model)

/home/johny02/anaconda3/envs/local_rag_chat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-17 12:47:25,714 [INFO] sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2024-12-17 12:47:44,360 [INFO] sentence_transformers.SentenceTransformer: 2 prompts are loaded, with the keys: ['query', 'text']


In [ ]:
nodes = loader.fit('./local_rag_chat/eval/data/paul_graham/paul_graham_essay.txt')

In [64]:
len(nodes)

39

In [10]:
from llama_index.core import Settings
Settings.llm = llm.get_llm()

In [45]:
from local_rag_chat.core.retrievers.hybrid_retriever import HybridRetriever
hybrid_retriever = HybridRetriever(nodes=nodes,embed_model=embed_model,top_k=5)

Batches: 100%|████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


In [46]:
retrieved_nodes = hybrid_retriever.retrieve("What did the author do growing up?")

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 28.50it/s]


We'll try out retrieval over a simple dataset.

In [41]:
from llama_index.core.response.notebook_utils import display_source_node

for node in retrieved_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** 9ba7e089-d70a-4b53-a318-a5761ad84d5f<br>**Similarity:** 0.016666666666666666<br>**Text:** What I Worked On February 2021 Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.
The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.<br>

**Node ID:** 0431753d-36de-47fe-95be-50db5ddd3334<br>**Similarity:** 0.016666666666666666<br>**Text:** Now you not only had colleagues, but colleagues who understood the problems you were facing and could tell you how they were solving them.
As YC grew, we started to notice other advantages of scale. The alumni became a tight community, dedicated to helping one another, and especially the current  batch, whose shoes they remembered being in. We also noticed that the startups were becoming one another's customers. We used to refer jokingly to the "YC GDP," but as YC grows this becomes less and less of a joke.
Now lots of startups get their initial set of customers almost entirely from among their batchmates.
I had not originally intended YC to be a full-time job. I was going to do three things: hack, write essays, and work on YC. As YC grew, and I grew more excited about it, it started to take up a lot more than a third of my attention. But for the first few years I was still able to work on other things.
In the summer of 2006, Robert and I started working on a new version of Arc. Thi...<br>

**Node ID:** 7e6e3828-6619-45e4-88a9-9f9a00c4706e<br>**Similarity:** 0.01639344262295082<br>**Text:** It's called Yorkville, and that was my new home. Now I was a New York artist — in the strictly technical sense of making paintings and living in New York.
I was nervous about money, because I could sense that Interleaf was on the way down. Freelance Lisp hacking work was very rare, and I didn't want to have to program in another language, which in those days would have meant C++ if I was lucky. So with my unerring nose for financial opportunity, I decided to write another book on Lisp. This would be a popular book, the sort of book that could be used as a textbook. I imagined myself living frugally off the royalties and spending all my time painting.<br>

### Build an Evaluation dataset of (query, context) pairs

In [17]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)

In [47]:
qa_dataset = generate_question_context_pairs(
    nodes, num_questions_per_chunk=2
)

100%|███████████████████████████████████████████| 39/39 [14:41<00:00, 22.61s/it]


In [49]:
list(qa_dataset.queries.values())[0]

'Here are two potential questions based on the context information:'

In [50]:
qa_dataset.save_json("pg_eval_dataset.json")

In [ ]:
# [optional] load
qa_dataset = EmbeddingQAFinetuneDataset.from_json("pg_eval_dataset.json")

In [85]:
from llama_index.core.evaluation import RetrieverEvaluator
metrics = ["hit_rate", "mrr"]
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=hybrid_retriever
)

In [88]:
list(qa_dataset.queries.items())[0]

('bb32d546-4d02-4b1c-b890-4476d8930efe',
 "What type of computer was used by the narrator's school district, and where was it located?")

In [92]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 33.46it/s]

Query: What type of computer was used by the narrator's school district, and where was it located?
Metrics: {'hit_rate': 1.0, 'mrr': 1.0}



In [93]:
# try it out on an entire dataset
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 24.69it/s]


In [96]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

In [97]:
display_results("hybrid eval", eval_results)

,retrievers,hit_rate,mrr
0,hybrid eval,0.794872,0.735043


## HybridRetriever without SemanticSplitter

In [6]:
from local_rag_chat.core.loaders.simple_loader import SimpleLoader
loader = SimpleLoader()

In [11]:
from local_rag_chat.core.embeddings.embedding_manager import EmbeddingManager

embed_model = EmbeddingManager(model='BAAI/bge-small-en-v1.5').get_embedding()

/home/johny02/anaconda3/envs/local_rag_chat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-18 15:22:52,866 [INFO] sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2024-12-18 15:23:04,465 [INFO] sentence_transformers.SentenceTransformer: 2 prompts are loaded, with the keys: ['query', 'text']


In [7]:
nodes = loader.fit('./local_rag_chat/eval/data/paul_graham/paul_graham_essay.txt')

In [12]:
from local_rag_chat.core.retrievers.hybrid_retriever import HybridRetriever
hybrid_retriever = HybridRetriever(nodes=nodes,embed_model=embed_model,top_k=5)

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.75it/s]


In [13]:
retrieved_nodes = hybrid_retriever.retrieve("What did the author do growing up?")

Batches: 100%|██████████| 1/1 [00:00<00:00, 29.15it/s]


In [14]:
from llama_index.core.response.notebook_utils import display_source_node

for node in retrieved_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** 17837ef9-6a60-4c57-9e17-8e492036675f<br>**Similarity:** 0.016666666666666666<br>**Text:** What I Worked On February 2021 Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.
The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.
The language we used was an early version of Fortran.
You had to type programs on punch cards, then stack them in the ...<br>

**Node ID:** 61d1f2fb-1a23-4230-a79a-14aa64d27763<br>**Similarity:** 0.016666666666666666<br>**Text:** If we'd been a startup I was advising at Y Combinator, I would have said: Stop being so stressed out, because you're doing fine. You're growing 7x a year. Just don't hire too many more people and you'll soon be profitable, and then you'll control your own destiny.
Alas I hired lots more people, partly because our investors wanted me to, and partly because that's what startups did during the Internet Bubble. A company with just a handful of employees would have seemed amateurish. So we didn't reach breakeven until about when Yahoo bought us in the summer of 1998.
Which in turn meant we were at the mercy of investors for the entire life of the company. And since both we and our investors were noobs at startups, the result was a mess even by startup standards.
It was a huge relief when Yahoo bought us. In  principle our Viaweb stock was valuable. It was a share in a business that was profitable and growing rapidly. But it didn't feel very valuable to me; I had no idea how to value a bu...<br>

**Node ID:** af908143-471a-45d9-bf85-9286b97e874f<br>**Similarity:** 0.01639344262295082<br>**Text:** Afterward I put a postscript file of this talk online, on paulgraham.com, which I'd created years before using Viaweb but had never used for anything. In one day it got 30,000 page views. What on earth had happened?
The referring urls showed that someone had posted it on Slashdot. [10] Wow, I thought, there's an audience. If I write something and put it on the web, anyone can read it.
That may seem obvious now, but it was surprising then. In the print era there was a narrow channel to readers, guarded by fierce monsters known as editors.
The only way to get an audience for anything you wrote was to get it published as a book, or in a newspaper or magazine. Now anyone could publish anything.
This had been possible in principle since 1993, but not many people had realized it yet. I had been intimately involved with building the infrastructure of the web for most of that time, and a writer as well, and it had taken me 8 years to realize it. Even then it took me several years to underst...<br>

In [15]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)

In [16]:
qa_dataset = generate_question_context_pairs(
    nodes, num_questions_per_chunk=2
)

100%|██████████| 42/42 [17:26<00:00, 24.91s/it]


In [17]:
qa_dataset.queries = {key:value for key, value in qa_dataset.queries.items() if "Here are" not in value}

In [19]:
from llama_index.core.evaluation import RetrieverEvaluator
metrics = ["hit_rate", "mrr"]
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=hybrid_retriever
)

In [20]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.44it/s]

Query: What was the primary input method used to program the IBM 1401, and what were some of its limitations?
Metrics: {'hit_rate': 1.0, 'mrr': 1.0}



In [21]:
# try it out on an entire dataset
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

Batches: 100%|██████████| 1/1 [00:00<00:00, 17.10it/s]


In [22]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

In [23]:
display_results("hybrid without semantic split", eval_results)

,retrievers,hit_rate,mrr
0,hybrid without semantic split,0.952381,0.869048
